## Imports

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np

from dataset.dataset import MultiModalDataset
from mmcv_model.mmcv_csn import ResNet3dCSN
from mmcv_model.scheduler import GradualWarmupScheduler

from model.multimodal_neck import MultiModalNeck
from model.simple_head import SimpleHead
from model.flow_autoencoder import FlowAutoencoder

/home/myuser1/miniconda3/envs/wlasl/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Model

In [4]:
# Custom multimodal model
rgb_backbone = ResNet3dCSN(
    pretrained2d=False,
    # pretrained=None,
    pretrained='https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth',
    depth=50,
    with_pool2=False,
    bottleneck_mode='ir',
    norm_eval=True,
    zero_init_residual=False,
    bn_frozen=True
)

# rgb_backbone.init_weights()
print('Loading rgb backbone checkpoint...')
rgb_checkpoint = torch.load('rgb_backbone.pth')
rgb_backbone.load_state_dict(rgb_checkpoint)
del rgb_checkpoint

flow_backbone = ResNet3dCSN(
    pretrained2d=False,
    # pretrained=None,
    pretrained='https://download.openmmlab.com/mmaction/recognition/csn/ircsn_from_scratch_r50_ig65m_20210617-ce545a37.pth',
    depth=50,
    with_pool2=False,
    bottleneck_mode='ir',
    norm_eval=True,
    zero_init_residual=False,
    bn_frozen=True
)

# flow_backbone.init_weights()
print('Loading flow backbone checkpoint...')
flow_checkpoint = torch.load('flow_backbone.pth')
flow_backbone.load_state_dict(flow_checkpoint)
del flow_checkpoint

print('Backbones loaded successfully.')

neck = MultiModalNeck()
head = SimpleHead(num_classes=400,
                  in_channels=4096,
                  dropout_ratio=0.5,
                  init_std=0.01)

head.init_weights()

model = FlowAutoencoder(rgb_backbone=rgb_backbone,
                        flow_backbone=flow_backbone,
                        neck=neck,
                        head=head)

Loading rgb backbone checkpoint...
Loading flow backbone checkpoint...
Backbones loaded successfully.


In [6]:
for name, para in flow_backbone.named_parameters():
    para.requires_grad = False

In [7]:
for name, para in model.flow_backbone.named_parameters():
    print("-"*20)
    print(f"name: {name}")
    print("values: ")
    print(para)

--------------------
name: conv1.conv.weight
values: 
Parameter containing:
tensor([[[[[ 3.0491e-03,  1.1189e-02,  1.3755e-02,  ...,  3.1400e-02,
             7.2105e-03,  1.4299e-02],
           [ 5.4376e-03, -5.0222e-03, -3.5202e-04,  ...,  2.6585e-02,
             1.4659e-04,  6.6237e-03],
           [ 5.3720e-03,  2.6433e-03,  7.4447e-03,  ...,  3.1563e-02,
            -1.8138e-03,  1.0702e-03],
           ...,
           [ 7.9799e-03,  1.6930e-02, -6.9132e-03,  ...,  2.1261e-02,
             1.3699e-02,  7.2404e-04],
           [ 4.1319e-03,  1.1094e-02,  4.6996e-03,  ...,  1.6642e-02,
             4.8081e-03,  1.1904e-02],
           [ 2.1023e-03,  1.3441e-02,  1.7854e-03,  ...,  4.8000e-03,
            -4.3225e-03,  5.1993e-04]],

          [[ 9.9654e-04, -1.3581e-02,  5.5273e-02,  ...,  7.2960e-02,
            -3.0251e-02,  2.9012e-03],
           [ 6.4911e-03, -3.9925e-02,  6.9921e-02,  ...,  3.1962e-02,
            -1.9011e-02,  9.9545e-03],
           [-2.4677e-03, -5.0351e-